In [22]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec

In [75]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [112]:
import spacy

In [2]:
!nvidia-smi

Tue Jan  9 10:23:19 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 528.25       Driver Version: 528.25       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   33C    P4    14W /  35W |      0MiB /  6141MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!nvidia-smi

Mon Jan 22 21:59:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.33                 Driver Version: 546.33       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4050 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   41C    P8               1W / 130W |     81MiB /  6141MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
data = pd.read_csv("Restaurant reviews.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Restaurant  10000 non-null  object 
 1   Reviewer    9962 non-null   object 
 2   Review      9955 non-null   object 
 3   Rating      9962 non-null   object 
 4   Metadata    9962 non-null   object 
 5   Time        9962 non-null   object 
 6   Pictures    10000 non-null  int64  
 7   7514        1 non-null      float64
dtypes: float64(1), int64(1), object(6)
memory usage: 625.1+ KB


In [4]:
data = data.drop(['Restaurant', 'Reviewer','Metadata', 'Time', 'Pictures', '7514'], axis=1)

In [5]:
data = data.dropna()

In [6]:
data = data[data.Rating != 'Like']

In [65]:
data.Rating.unique()

array(['5', '4', '1', '3', '2', '3.5', '4.5', '2.5', '1.5'], dtype=object)

In [68]:
data['Rating'] = data['Rating'].astype('float64')

In [8]:
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

In [53]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anantha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Anantha\AppData\Roaming\nltk_data...


True

In [9]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
tag_dict = {'J': wordnet.ADJ,'N': wordnet.NOUN,'V': wordnet.VERB,'R': wordnet.ADV}

## Sentiment Analysis

In [24]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [26]:
sid_obj = SentimentIntensityAnalyzer()

In [36]:
lol = data['Review'][9997]
print(lol)
sid_obj.polarity_scores(lol)

Bad rating is mainly because of "Chicken Bone found in Veg food" and is not acceptable for a Vegetarian.

To satiate Cravings for chinese food we headed to Chinese Pavilion and there was also a voucher available in Little app for Luncheon tray for Rs 310 (Veg) [well do not bother for the voucher as the price for Veg Luncheon tray is Rs 275/- + taxes in the restaurant itself).

The ambiance is decent and since it was noon, not many tables occupied so it was pretty quite.

It was Thrusday special (Veg):
Hot and Sour soup: was just OK - well it had sprouts which were crunchy in your mouth: something different.
Kimchi - salad (personally i'm never fond of it)
Potato in plum sauce - this was very good and well made
Brocolli, Baby corn, Mushroom in spicy sauce - this dish lived to its name 'too spicy'; but was fine with fried rice
Veg hakka noodles - was good
Veg fried rice

Overall the food tasted good until a piece of chicken BONE came into my friend's (he is pure vegetarian) mouth while e

{'neg': 0.048, 'neu': 0.811, 'pos': 0.141, 'compound': 0.978}

In [61]:
def senti(sentence):
    di = sid_obj.polarity_scores(sentence)
    if di['compound'] >= 0.9:
        return "Positive"
    elif di['compound'] <= - 0.9:
        return "Negative"
    else:
        return "Neutral"

In [62]:
data['Sentiment'] = data['Clean_Review'].apply(senti)

In [40]:
senti = data[(data['Sentiment'] == "Positive") & (data['Rating'] == '1')]
senti.reset_index(drop=True, inplace=True)

In [63]:
data

,Review,Rating,Clean_Review,Sentiment
0,"The ambience was good, food was quite good . h...",5,ambience good food quite good saturday lunch c...,Positive
1,Ambience is too good for a pleasant evening. S...,5,ambience good pleasant even service prompt foo...,Positive
2,A must try.. great food great ambience. Thnx f...,5,must try great food great ambience thnx servic...,Positive
3,Soumen das and Arun was a great guy. Only beca...,5,soumen das arun great guy behavior sincerety g...,Neutral
4,Food is good.we ordered Kodi drumsticks and ba...,5,food goodwe order kodi drumstick basket mutton...,Positive
...,...,...,...,...
9995,Madhumathi Mahajan Well to start with nice cou...,3,madhumathi mahajan well start nice courteous s...,Positive
9996,This place has never disappointed us.. The foo...,4.5,place never disappoint u food courteous staff ...,Positive
9997,"Bad rating is mainly because of ""Chicken Bone ...",1.5,bad rating mainly chicken bone find veg food a...,Positive
9998,I personally love and prefer Chinese Food. Had...,4,personally love prefer chinese food couple tim...,Positive


In [ ]:
t1 = ['5', '4', '1', '3', '2', '3.5', '4.5', '2.5', '1.5']
t2 = ['Positive', 'Negative','Neutral']


In [30]:
#URLs present
for i in range(9954):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    if re.search(url_pattern, data.iloc[i,0]):
        print(i)

371
2596


In [17]:
import string
def preprocess_text(text):
    text = str(text).lower()
    clean_text = re.sub('https?://\S+|www\.\S+', '', text) #URLs
    clean_text = re.sub('[%s]' % re.escape(string.punctuation), '', clean_text) #Punctuations
    clean_text = re.sub('\n', '', clean_text) #Backslash n
    clean_text = [word for word in clean_text.split(' ') if word not in stop_words] #Remove stopwords
    clean_text=" ".join(clean_text)
    tokens = clean_text.split()
    pos_tags = nltk.pos_tag(tokens)
    wordnet_tags = [(token, tag_dict.get(tag[0].upper(), wordnet.NOUN)) for token, tag in pos_tags]
    tokens = [lemmatizer.lemmatize(token, tag) for token, tag in wordnet_tags] #Lemmatize
    return " ".join(tokens)

In [18]:
data['Clean_Review'] = data['Review'].apply(preprocess_text)

In [20]:
data['Clean_Review'][0]

'ambience good food quite good saturday lunch cost effective good place sate brunch one also chill friend parentswaiter soumen das really courteous helpful'

In [21]:
data

,Review,Rating,Clean_Review
0,"The ambience was good, food was quite good . h...",5,ambience good food quite good saturday lunch c...
1,Ambience is too good for a pleasant evening. S...,5,ambience good pleasant even service prompt foo...
2,A must try.. great food great ambience. Thnx f...,5,must try great food great ambience thnx servic...
3,Soumen das and Arun was a great guy. Only beca...,5,soumen das arun great guy behavior sincerety g...
4,Food is good.we ordered Kodi drumsticks and ba...,5,food goodwe order kodi drumstick basket mutton...
...,...,...,...
9995,Madhumathi Mahajan Well to start with nice cou...,3,madhumathi mahajan well start nice courteous s...
9996,This place has never disappointed us.. The foo...,4.5,place never disappoint u food courteous staff ...
9997,"Bad rating is mainly because of ""Chicken Bone ...",1.5,bad rating mainly chicken bone find veg food a...
9998,I personally love and prefer Chinese Food. Had...,4,personally love prefer chinese food couple tim...


In [ ]:
vectorizer = CountVectorizer()
x_train_bow = vectorizer.fit_transform(x_train)
x_test_bow = vectorizer.fit_transform(x_test)

In [55]:
string.punctuation
import string
for i in range(9954):
    url_pattern = re.compile('\[.*?\]')
    if re.search(url_pattern, data.iloc[i,0]):
        print(i)

1613
2549
5629
5887
7177
7681
8760
9237


In [48]:
re.sub('[%s]' % re.escape(string.punctuation), '', data.iloc[5,0])

'Ambiance is good service is good food is aPradeecp and subro best service\nFood is good Papiya good hostess and ur caption very good This is 4star restaurant'

In [59]:
data.iloc[2549,0]
# data.iloc[1,0]

"I have been to this place twice. My first experience wasn't that great as I found the food items to be sub-par. So I quickly went to a different place. After 2 months I recently went again for an office party with ~40 people and it turned out to be a better experience. Starters [chilli chicken, fries, cheese poppers, some fish] were fine but nothing extra-ordinary, ambience was a bit crazy with random groups dancing here & there, DJ was average, Biryani was good I heard but did not try it myself. Pizza was a big let down both in taste & presentation, it was served cut into small rectangular pieces. Service was decent. Staff courteous. Good interiors.\nOverall - 3.5 considering the prices. Recommended for group drinking sessions, maybe a bit of dancing too."

In [71]:
data['Sentiment'] = data['Rating'].apply(lambda x: 'Positive' if x > 3 else('Neutral' if x==3 else 'Negative'))

In [123]:
data['Clean_Review'][0]

'ambience good food quite good saturday lunch cost effective good place sate brunch one also chill friend parentswaiter soumen das really courteous helpful'

In [88]:
# sentence_bow = vectorizer.fit_transform(data['Clean_Review'])
x_train, x_test, y_train, y_test = train_test_split(data['Clean_Review'], data['Sentiment'], test_size=0.2, random_state=42)

### Bag of Words

In [ ]:
vectorizer = CountVectorizer()
x_train_bow = vectorizer.fit_transform(x_train)
x_test_bow = vectorizer.transform(x_test)

In [93]:
x_test_bow

<1991x20598 sparse matrix of type '<class 'numpy.int64'>'
	with 42926 stored elements in Compressed Sparse Row format>

In [79]:
print(x_train_bow)

  (0, 1750)	1
  (0, 9414)	3
  (0, 8464)	1
  (0, 17355)	1
  (0, 18665)	1
  (0, 12868)	1
  (0, 5407)	1
  (0, 7221)	1
  (0, 16371)	1
  (0, 18633)	1
  (0, 3791)	1
  (0, 15088)	1
  (0, 1613)	1
  (0, 4583)	1
  (0, 8814)	1
  (0, 15791)	1
  (0, 19835)	1
  (0, 5954)	1
  (0, 17597)	1
  (0, 5498)	1
  (0, 10274)	1
  (1, 1750)	1
  (1, 9414)	3
  (1, 8464)	1
  (1, 19835)	1
  :	:
  (9953, 18122)	2
  (9953, 4512)	2
  (9953, 13713)	1
  (9953, 7236)	4
  (9953, 13858)	1
  (9953, 18483)	1
  (9953, 8215)	2
  (9953, 1590)	1
  (9953, 11241)	1
  (9953, 4409)	1
  (9953, 13642)	1
  (9953, 379)	1
  (9953, 16094)	1
  (9953, 4347)	1
  (9953, 21518)	1
  (9953, 2699)	1
  (9953, 22188)	3
  (9953, 10018)	2
  (9953, 6843)	1
  (9953, 12057)	1
  (9953, 1890)	1
  (9953, 12876)	2
  (9953, 23618)	1
  (9953, 21231)	1
  (9953, 16968)	1


In [96]:
clf_bow = RandomForestClassifier()
clf_bow.fit(x_train_bow, y_train)

RandomForestClassifier()

In [99]:
y_pred_bow = clf_bow.predict(x_test_bow)
accuracy_bow = accuracy_score(y_test, y_pred_bow)
print("Accuracy using Bag of Words:", accuracy_bow)

Accuracy using Bag of Words: 0.8061275740833752


### Tf-idf vectorizer

In [82]:
vectorizer_tfidf = TfidfVectorizer()
x_train_tfidf = vectorizer_tfidf.fit_transform(x_train)
x_test_tfidf = vectorizer_tfidf.transform(x_test)

In [87]:
x_train_tfidf

<7963x20598 sparse matrix of type '<class 'numpy.float64'>'
	with 185591 stored elements in Compressed Sparse Row format>

In [84]:
print(x_train_tfidf)

  (0, 15741)	0.36806107900618523
  (0, 17083)	0.24453671672328398
  (0, 1424)	0.2212043459233207
  (0, 16649)	0.31437173991812695
  (0, 6990)	0.29114134852937423
  (0, 2677)	0.35563732692373534
  (0, 9176)	0.2326202244485168
  (0, 14013)	0.22554950519313405
  (0, 16450)	0.3068779530522159
  (0, 15009)	0.2810175277309669
  (0, 8910)	0.26224305847721296
  (0, 19027)	0.31437173991812695
  (1, 6809)	0.20166559259397213
  (1, 7521)	0.17042801624152076
  (1, 12502)	0.36426905967229445
  (1, 6299)	0.42105087855331697
  (1, 2014)	0.2789337589155798
  (1, 7546)	0.42105087855331697
  (1, 15074)	0.28076076257627003
  (1, 1549)	0.42105087855331697
  (1, 12405)	0.1437964208252754
  (1, 7233)	0.0844000911506514
  (1, 18012)	0.1162097199767859
  (1, 8037)	0.24431850097073413
  (1, 14013)	0.09007853568055495
  :	:
  (7961, 1424)	0.14818953395657006
  (7962, 17717)	0.28624470773646343
  (7962, 16367)	0.28624470773646343
  (7962, 46)	0.28624470773646343
  (7962, 19093)	0.28624470773646343
  (7962, 18849

In [94]:
clf_bow = RandomForestClassifier()
clf_bow.fit(x_train_tfidf, y_train)

RandomForestClassifier()

In [95]:
y_pred_tfidf = clf_bow.predict(x_test_tfidf)
accuracy_tfidf = accuracy_score(y_test, y_pred_tfidf)
print("Accuracy using TF IDF:", accuracy_tfidf)

Accuracy using TF IDF: 0.7990959316926167


### GloVe

In [115]:
nlp = spacy.load("en_core_web_md")

In [117]:
def get_average_vector(sentence):
    doc = nlp(sentence)
    vectors = [token.vector for token in doc if token.has_vector]
    if not vectors:
        return [0] * nlp.vocab.vectors.shape[1]
    return sum(vectors) / len(vectors)

In [118]:
data['GloVe_Embedding'] = data['Clean_Review'].apply(get_average_vector)

In [121]:
data['GloVe_Embedding'][0]

array([ 4.33147848e-02, -1.39705881e-01, -2.58184171e+00, -6.98054552e-01,
        1.60200250e+00, -1.66281447e-01,  1.14463198e+00,  2.09158897e+00,
       -1.64169586e+00,  1.20689249e+00,  4.62465715e+00,  7.18688846e-01,
       -2.45166898e+00,  1.09102809e+00,  6.11897171e-01,  5.04617631e-01,
        2.06365180e+00, -1.66012096e+00, -4.13142383e-01, -9.03644025e-01,
       -3.87937188e-01,  1.32229710e+00, -4.10629123e-01, -1.03399241e+00,
       -2.04100490e-01, -1.62850499e-01, -3.66693044e+00, -9.00457501e-02,
       -1.06157076e+00,  1.54027832e+00,  1.83303773e+00, -8.92971039e-01,
       -9.31362867e-01, -2.01225281e+00,  1.32793045e+00, -6.96671963e-01,
        8.09838176e-02,  8.57670367e-01,  1.29544389e+00,  1.17430127e+00,
       -7.30051398e-01,  1.46843505e+00,  1.34479201e+00, -9.73996103e-01,
        2.50586309e-02,  1.87412536e+00,  1.67042482e+00, -2.44603300e+00,
       -2.69448817e-01,  1.67907357e+00,  7.57770389e-02,  8.17190528e-01,
       -5.54659009e-01, -

### Word2Vec

In [103]:
data['Clean_Review'] = data['Clean_Review'].astype(str)

data['Tokenized_Review'] = data['Review'].apply(lambda x: [word for word in word_tokenize(x) if word.isalnum()])

word2vec_model = Word2Vec(sentences=data['Tokenized_Review'], vector_size=100, window=5, min_count=1, workers=4)

def get_average_vector(words, model, vector_size):
    vectors = [model.wv[word] for word in words if word in model.wv]
    if not vectors:
        return [0] * vector_size
    return sum(vectors) / len(vectors)

data['Word2Vec_Embedding'] = data['Tokenized_Review'].apply(lambda x: get_average_vector(x, word2vec_model, vector_size=100))

In [110]:
data['Word2Vec_Embedding'][0]

array([ 0.16281794,  0.18560486, -0.4948339 ,  0.30996305, -0.15118147,
       -0.60909635,  0.48877642,  1.0412368 , -0.45515212, -0.34917292,
       -0.3048055 , -0.39448348,  0.11992119,  0.64151144,  0.19303434,
        0.04732047,  0.29674637, -0.27253875, -0.36874992, -0.4507706 ,
        0.31425214,  0.08282745,  0.3853743 , -0.28625858,  0.01253318,
       -0.17420126, -0.39951968, -0.19729508, -0.13099825,  0.04684052,
        0.67377174, -0.44157666,  0.5087766 , -0.48944032,  0.06267449,
        0.69517225,  0.23146817,  0.30889186, -0.16194928, -0.65044767,
       -0.14455879, -0.3633359 , -0.23871242,  0.07876381, -0.05787921,
        0.08262037, -0.13247597,  0.02350674,  0.17410804,  0.16511358,
       -0.34981656, -0.4056856 , -0.09174228, -0.7354576 , -0.20990947,
       -0.26268208,  0.15415677, -0.13055822, -0.20178467,  0.6203513 ,
        0.30198383, -0.29908833,  0.8444529 ,  0.10073119, -0.15246132,
        0.5209164 ,  0.37815687,  0.70070237, -0.78060955,  0.51

In [106]:
x_train_w2v, x_test_w2v, y_train, y_test = train_test_split(data['Word2Vec_Embedding'], data['Sentiment'], test_size=0.2, random_state=42)